In [1]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM,Dense
from tensorflow.keras.callbacks import TensorBoard

In [2]:
import os
from PIL import Image
from tqdm import tqdm
import numpy as np
import pandas as pd


In [41]:
data_path = "D:\\project\\dataset\\gen_land_marks_dataset\\FrameLM_.csv"


data = pd.read_csv(data_path)
data['sign_n'] = data['sign'].astype('category').cat.codes

data

,1x0,1y0,1z0,1x1,1y1,1z1,1x2,1y2,1z2,1x3,...,1z18,1x19,1y19,1z19,1x20,1y20,1z20,frame,sign,sign_n
0,0.792746,0.990288,-8.256451e-07,0.734886,0.911677,-0.003520,0.711027,0.794658,-0.004054,0.700255,...,-0.029235,0.845639,0.630521,-0.024903,0.818636,0.681482,-0.014796,1,YES,1
1,0.799704,0.992832,-8.117081e-07,0.739026,0.895418,0.003122,0.713416,0.788441,0.004467,0.703935,...,-0.023513,0.850635,0.628874,-0.019362,0.824483,0.688885,-0.008970,2,YES,1
2,0.799565,0.993079,-8.044564e-07,0.740635,0.893880,0.001804,0.713768,0.788145,0.001861,0.704011,...,-0.025401,0.853576,0.628154,-0.021274,0.826121,0.687863,-0.011128,3,YES,1
3,0.798942,0.993288,-8.204698e-07,0.741771,0.896844,-0.000105,0.716491,0.788728,-0.000861,0.706110,...,-0.026144,0.853186,0.628007,-0.021998,0.826591,0.688785,-0.011470,4,YES,1
4,0.799430,0.997525,-8.401622e-07,0.740387,0.895666,0.004254,0.715142,0.788283,0.005445,0.705099,...,-0.022941,0.853379,0.629244,-0.018652,0.826039,0.689840,-0.008591,5,YES,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
595,0.626633,0.777197,6.241039e-08,0.541895,0.721393,0.040732,0.497637,0.648182,0.036825,0.451215,...,-0.105235,0.519936,0.641065,-0.093330,0.554812,0.663161,-0.080579,16,NO,0
596,0.636006,0.802810,2.072078e-07,0.552586,0.742736,0.011503,0.501454,0.657794,-0.000170,0.457878,...,-0.102227,0.521275,0.638136,-0.092558,0.553049,0.663608,-0.078551,17,NO,0
597,0.620894,0.760771,3.646648e-07,0.547273,0.718012,0.008708,0.502508,0.641607,-0.003569,0.458056,...,-0.108910,0.528940,0.636936,-0.103565,0.561864,0.664369,-0.092096,18,NO,0
598,0.646680,0.796605,2.927184e-07,0.561233,0.734687,0.016453,0.513528,0.648054,0.006548,0.469913,...,-0.096801,0.533951,0.625852,-0.086504,0.564234,0.656142,-0.072171,19,NO,0


In [42]:
keypints_array = data.values[:,:63].reshape(30, 20, 21,3)
lable_array = data.values[:,-1].reshape(30, 20, 1)

In [43]:
keypints_array.shape

(30, 20, 21, 3)

In [18]:
#keypints_array = keypints_array.reshape(80, 15, 126)

In [44]:
lable_array_ = np.zeros((30, 2))
for i in range(keypints_array.shape[0]):
    lable_array_[i][lable_array[i][0][0]] = 1.

In [45]:
lable_array_

array([[0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.]])

In [47]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(keypints_array, lable_array_, test_size=0.1, random_state=42)

In [46]:
keypints_array = keypints_array.astype('float32')

In [37]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
model = Sequential()

model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(20, 21,3)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(2, activation='softmax'))

In [48]:
model.compile(optimizer = 'Adam',loss='categorical_crossentropy',metrics=['categorical_accuracy'])

In [49]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 18, 19, 32)        896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 9, 9, 32)         0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 7, 7, 64)          18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 3, 3, 64)         0         
 2D)                                                             
                                                                 
 flatten (Flatten)           (None, 576)               0         
                                                                 
 dense_9 (Dense)             (None, 128)              

In [51]:
model.fit(X_train,y_train,epochs = 200)
model.summary()

Epoch 1/200
1/1 [==============================] - 0s 12ms/step - loss: 9.9604e-06 - categorical_accuracy: 1.0000
Epoch 2/200
1/1 [==============================] - 0s 11ms/step - loss: 9.8059e-06 - categorical_accuracy: 1.0000
Epoch 3/200
1/1 [==============================] - 0s 10ms/step - loss: 9.6469e-06 - categorical_accuracy: 1.0000
Epoch 4/200
1/1 [==============================] - 0s 10ms/step - loss: 9.4880e-06 - categorical_accuracy: 1.0000
Epoch 5/200
1/1 [==============================] - 0s 11ms/step - loss: 9.3467e-06 - categorical_accuracy: 1.0000
Epoch 6/200
1/1 [==============================] - 0s 11ms/step - loss: 9.1966e-06 - categorical_accuracy: 1.0000
Epoch 7/200
1/1 [==============================] - 0s 13ms/step - loss: 9.0597e-06 - categorical_accuracy: 1.0000
Epoch 8/200
1/1 [==============================] - 0s 11ms/step - loss: 8.9317e-06 - categorical_accuracy: 1.0000
Epoch 9/200
1/1 [==============================] - 0s 12ms/step - loss: 8.7948e-06 - cat

In [55]:
model.predict(X_test)

1/1 [==============================] - 0s 20ms/step


array([[9.9999976e-01, 2.1595901e-07],
       [1.2741646e-07, 9.9999988e-01],
       [9.9999905e-01, 9.1920612e-07]], dtype=float32)

In [56]:
y_test

array([[1., 0.],
       [1., 0.],
       [1., 0.]])

In [172]:
dataset[0]

(tensor([[ 0.8468,  0.9027,  0.7710,  ...,  0.0000,  0.0000,  0.0000],
         [ 0.8485,  0.9221,  0.7710,  ...,  0.0000,  0.0000,  0.0000],
         [ 0.8405,  0.9277,  0.7710,  ...,  0.0000,  0.0000,  0.0000],
         ...,
         [ 0.0989,  0.9731,  0.7710,  ...,  0.9065,  0.5294, -0.0606],
         [ 0.1095,  0.9907,  0.7710,  ...,  0.9556,  0.5438, -0.0664],
         [ 0.1116,  1.0000,  0.7710,  ...,  0.9860,  0.5429, -0.0835]]),
 0.0)

In [171]:
#dataset[0]
lable_array_[0]

0.0

In [183]:

#test_dataset = MyDataset(test_data)
TrainDataLoader = DataLoader(dataset, batch_size=1)
#TestDataLoader = DataLoader(v, batch_size=64)

In [187]:
class LSTMmodel(nn.Module):
    def __init__(self, num_classes ):
        super().__init__()
        self.lstm = nn.LSTM(input_size=126, hidden_size=500, num_layers=1, batch_first=True)
        self.dropout = nn.Dropout(0.2)
        self.linear = nn.Linear(500, num_classes)
    def forward(self, x):
        x, _ = self.lstm(x)
        x = self.dropout(x)
        x = self.linear(x)
        return x

model = LSTMmodel(4)